## Load the libraries

In [3]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os, argparse
import cv2, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
from sklearn.externals import joblib
from keras import backend as K
from keras.utils.vis_utils import plot_model
K.set_image_data_format('channels_first')
#K.set_image_dim_ordering('th')

## Path to the models and weights files



In [2]:
# File paths for the model, all of these except the CNN Weights are 
# provided in the repo, See the models/CNN/README.md to download VGG weights
VQA_model_file_name      = 'models/VQA/VQA_MODEL.json'
VQA_weights_file_name   = 'models/VQA/VQA_MODEL_WEIGHTS.hdf5'
label_encoder_file_name  = 'models/VQA/FULL_labelencoder_trainval.pkl'
CNN_weights_file_name   = 'models/CNN/vgg16_weights.h5'


## Model Idea
This uses a classical CNN-LSTM  model like shown below, where Image features and language features are computed separately and combined together and a multi-layer perceptron is trained on the combined features.

#### So basically the model consists of the following components : 
    • CNN model - for extracting the image features 
    • Word Embedding - to convert word2vec using Glove 
    • VQA model - combine features from Image and Word Embeddings and runs a multiple layer perceptron


<img src="http://i.imgur.com/Za5P1ZZ.png">
[Source](http://arxiv.org/pdf/1505.00468v4.pdf)

# Image features 

## Pretrained VGG Net (VGG-16)

We used a pretrained image feature extraction model and did not build our own as we required good accuracy to build the fusion model. This domain of ML is very saturated and we decided to take an existing model upon which we will add layers to meet our needs. 
So we chose VGG-16 as it is very versatile, simple, relatively small and most importantly portable to use. 

Our focus was on building the VQA model compared to the already existing image feature extarction models, which perform very well. 

For reference here is the VGG 16 performance on ILSVRC-2012
<img src="http://www.robots.ox.ac.uk/~vgg/research/very_deep/images/table_ILSVRC.png">

## Compile the model

In [3]:
def get_image_model(CNN_weights_file_name):
    from models.CNN.VGG import VGG_16
    image_model = VGG_16(CNN_weights_file_name)
    image_model.layers.pop()
    image_model.layers.pop()
    # this is standard VGG 16 without the last two layers
    #remove the softmax and drop out layers
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    # one may experiment with "adam" optimizer, but the loss function for
    # this kind of task is pretty standard
    image_model.compile(optimizer=sgd, loss='categorical_crossentropy')
    return image_model

## Plot the Model

Visualizing the model in block diagram, to get a better idea of the below layers. 

In [4]:
model_vgg = get_image_model(CNN_weights_file_name)
plot_model(model_vgg, to_file='model_vgg.png')

<img src="model_vgg.png">

## Extract Image features


Extracting image features involves, taking a raw image, and running it through the model, until we reach the last layer. In this case our model is not 100% same as VGG Net, because we are not going to use the last two layer of the VGG. It is because the last layer of VGG Net is a 1000 way softmax and the second last layer is the Dropout.

Thus we are extracting the 4096 Dimension image features from VGG-16


In [5]:
def get_image_features(image_file_name):
    ''' Runs the given image_file to VGG 16 model and returns the 
    weights (filters) as a 1, 4096 dimension vector '''
    image_features = np.zeros((1, 4096))
    # Magic_Number = 4096  > Comes from last layer of VGG Model

    # Since VGG was trained as a image of 224x224, every new image
    # is required to go through the same transformation
    im = cv2.resize(cv2.imread(image_file_name), (224, 224))
    im = im.transpose((2,0,1)) # convert the image to RGBA

    
    # this axis dimension is required because VGG was trained on a dimension
    # of 1, 3, 224, 224 (first axis is for the batch size
    # even though we are using only one image, we have to keep the dimensions consistent
    im = np.expand_dims(im, axis=0) 

    image_features[0,:] = model_vgg.predict(im)[0]
    return image_features

# Word Embeddings

Now that we have extracted the image features, we now need a way to undertand the questions being asked.  
To do so the question has to be converted into some form of word embeddings.So we used a Word representation from Stanford called Glove. Glove reduces a given token into a 300 dimensional representation.

In [6]:
def get_question_features(question):
    ''' For a given question, a unicode string, returns the time series vector
    with each word (token) transformed into a 300 dimension representation
    calculated using Glove Vector '''
    word_embeddings = spacy.load('en_vectors_web_lg')
    tokens = word_embeddings(question)
    question_tensor = np.zeros((1, 30, 300))
    for j in xrange(len(tokens)):
        question_tensor[0,j,:] = tokens[j].vector
    return question_tensor

## Try the embeddings

Let's see the embeddings, and their usage with sample words like this -
1. Obama 
2. Putin
3. Banana
4. Monkey

In [7]:
word_embeddings = spacy.load('en', vectors='en_glove_cc_300_1m_vectors')

In [8]:
obama = word_embeddings(u"obama")
putin = word_embeddings(u"putin")
banana = word_embeddings(u"banana")
monkey = word_embeddings(u"monkey")

In [9]:
obama.similarity(putin)

0.5735777213479505

In [10]:
obama.similarity(banana)

0.4924444357063912

In [11]:
banana.similarity(monkey)

0.5471796768752016

As we can see, obama and putin are very similar in representation than obama and banana. This shows you there is some semantic knowledge of the tokens embedded in the 300 dimensional representation. We can do cool arithmatics with these word2vec like 'Queen' - 'King' + 'Boy' = 'Girl'. 

## VQA Model

VQA is a simple model which combines features from Image and Word Embeddings and runs a multiple layer perceptron, to get our desired output.

In [12]:
def get_VQA_model(VQA_model_file_name, VQA_weights_file_name):
    ''' Given the VQA model and its weights, compiles and returns the model '''

    # thanks the keras function for loading a model from JSON, this becomes
    # very easy to understand and work. Alternative would be to load model
    # from binary like cPickle but then model would be obfuscated to users
    vqa_model = model_from_json(open(VQA_model_file_name).read())
    vqa_model.load_weights(VQA_weights_file_name)
    vqa_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return vqa_model

In [13]:
model_vqa = get_VQA_model(VQA_model_file_name, VQA_weights_file_name)
plot_model(model_vqa, to_file='model_vqa.png')

<img src="model_vqa.png">

As it can be seen above the model also runs a 3 layered LSTM on the word embeddings. Then we feed the word embeddings directly to the merge layer
Also, four layers of fully connected layers might not be required to achieve a good enough results. But we settled on this model after some experimentation.
We decided to add the LSTM on the word embedding after reading the "Deeper LSTM and normalized CNN Visual Question Answering model" paper. 

## Lets test our hardwork now !

providing a test image and a question

In [14]:
image_file_name = 'web/static/test_images/test.jpg'
question = u"Is there a train in this picture?"

# <center> What vehicle is in the picture ? </center>

<img src="web/static/test_images/test.jpg">

In [15]:
# get the image features
image_features = get_image_features(image_file_name)

In [16]:
# get the question features
question_features = get_question_features(question)

In [28]:
y_output = model_vqa.predict([question_features, image_features])

# This task here is represented as a classification into a 1000 top answers
warnings.filterwarnings("ignore", category=DeprecationWarning)
labelencoder = joblib.load(label_encoder_file_name)
for label in reversed(np.argsort(y_output)[0,-5:]):
    print str(round(y_output[0,label]*100,2)).zfill(5), "% ", labelencoder.inverse_transform(label)

78.72 %  yes
21.28 %  no
000.0 %  girl
000.0 %  halloween
000.0 %  left
